In [1]:
## DATA MANIPULATION
import numpy as np, pandas as pd
import pickle

## CLASSIFICATION
from sklearn.base import clone
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from sklearn.pipeline import Pipeline
from sklearn.svm import LinearSVC
from sklearn.linear_model import LogisticRegression

from xgboost import XGBClassifier

### Load preprocessed data

In [2]:
f = open('../data/preprocessed.pkl','rb')
train, valid = pickle.load(f)
labels = train.columns[2:]
ys_train = train[labels]
ys_valid = valid[labels]

In [9]:
## COMBINE TOXIC CATEGORIES
y_train = ys_train.sum(axis=1)
y_valid = ys_valid.sum(axis=1)
y_train.loc[y_train>1] = 1
y_valid.loc[y_valid>1] = 1

### GloVe Vectorizer

In [14]:
## BUILD GLOVE EMBEDDINGS DICTIONARY
embeddings_dict = dict()
f = open(r'../data/glove.6B.300d.txt', encoding='utf8')
for line in f:
    values = line.split()
    word = values[0]
    vec = np.asarray(values[1:], dtype='float32')
    embeddings_dict[word] = vec
f.close()
print('Extracted {} word vectors'.format(len(embeddings_dict)))

Extracted 400000 word vectors


In [15]:
## SKLEARN COMPATIBLE GLOVE VECTORIZER TRANSFORMER 
class gloveVectorizer(object):
    def __init__(self, embeddings_dict):
        self.embeddings_dict = embeddings_dict
        # if a text is empty we should return a vector of zeros
        # with the same dimensionality as all the other vectors
        self.dim = len(embeddings_dict[next(iter(embeddings_dict))])

    def fit(self, X, y):
        return self
    
    def sentence2vec(self, s):
        '''
        Input:
        Sentence string

        Transformations:
        Get vector for each word -> Average vectors

        Output:
        Vector for sentence
        '''
        words = s.split()
        M = []
        for w in words:
            try:
                M.append(self.embeddings_dict[w])
            except:
                continue
        M = np.array(M)
        v = M.mean(axis=0)
        if type(v) != np.ndarray:
            return np.zeros(self.dim)
        return v # / np.sqrt((v ** 2).sum())

    def transform(self, X):
        X = X.apply(self.sentence2vec)
        return np.stack(X.values, axis=0)

In [16]:
glove = gloveVectorizer(embeddings_dict)
X_train = glove.transform(train['comment_text'])
X_valid = glove.transform(valid['comment_text'])

### Loop through models

In [22]:
## CREATE RESULTS TABLE
results = pd.DataFrame(columns=['Label', 'Accuracy', 'Recall', 'Precision', 'F1', 'Vectorizer', 'model'])

## CREATE MODELS
models = {
    'Logistic Regression': LogisticRegression(solver='saga',class_weight='balanced'),
    'SVM': LinearSVC(class_weight='balanced'),
    'XGBoost': XGBClassifier(n_estimators=100,
                        scale_pos_weight= sum(y_train==0) / sum(y_train==1),
                        n_jobs=-1)}
                      
## LOOP THROUGH MODELS
for m_label, model in models.items():
        m = clone(model) 

        # Fit model
        m.fit(X_train, y_train)
        
        # Get predictions
        preds = m.predict(X_valid)

        # Evaluate predictions
        acc, prec, recall, f1 = (accuracy_score(y_valid, preds), 
                                precision_score(y_valid, preds), 
                                recall_score(y_valid, preds), 
                                f1_score(y_valid, preds))
        
        # Save results to dataframe
        results = results.append({'Label': 'Toxic_Combined',
                                'Accuracy':acc,
                                'Recall':recall,
                                'Precision':prec,
                                'F1':f1,
                                'Vectorizer':'glove',
                                'model': m_label}, 
                                ignore_index = True)
        
        # print results
        print('{0} Results for {1} comments: Accuracy - {2:.2f}; Precision - {3:.2f}; Recall - {4:.2f}; F1 - {5:.2f}'.format(
                                        m_label,
                                        'Toxic_Combined', 
                                        acc, 
                                        prec, 
                                        recall,
                                        f1))

Logistic Regression Results for Toxic_Combined comments: Accuracy - 0.90; Precision - 0.49; Recall - 0.85; F1 - 0.63
SVM Results for Toxic_Combined comments: Accuracy - 0.90; Precision - 0.51; Recall - 0.85; F1 - 0.63
XGBoost Results for Toxic_Combined comments: Accuracy - 0.93; Precision - 0.66; Recall - 0.75; F1 - 0.70


In [23]:
results.to_csv('../artifacts/simple/glove.csv')